In [1]:
import zarr
import stackstac
import numpy as np
import pandas as pd
import xarray as xr
import geopandas as gpd
import planetary_computer as pc
from dask.distributed import Client
from shapely.geometry import mapping, box
from pystac_client import Client as pystac_client

#### Set up dask client

In [2]:
client = Client(
    threads_per_worker=1,
    memory_limit="4GB"
)
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 20
Total threads: 20,Total memory: 74.51 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:54646,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:54749,Total threads: 1
Dashboard: http://127.0.0.1:54787/status,Memory: 3.73 GiB
Nanny: tcp://127.0.0.1:54649,


2025-09-21 16:31:40,683 - distributed.dashboard.components.scheduler - ERROR - 'sub-f27b7da6d2b9a5674b7e1b29b406746b'
Traceback (most recent call last):
  File "c:\Users\geo1k\.local\share\mamba\envs\geodask\Lib\site-packages\distributed\utils.py", line 824, in wrapper
    return func(*args, **kwargs)
  File "c:\Users\geo1k\.local\share\mamba\envs\geodask\Lib\site-packages\distributed\dashboard\components\scheduler.py", line 2630, in update_layout
    x = max(xs[dep] for dep in dependencies[tg]) + 1
        ~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\geo1k\.local\share\mamba\envs\geodask\Lib\site-packages\distributed\dashboard\components\scheduler.py", line 2630, in <genexpr>
    x = max(xs[dep] for dep in dependencies[tg]) + 1
            ~~^^^^^
KeyError: 'sub-f27b7da6d2b9a5674b7e1b29b406746b'
2025-09-21 16:31:40,687 - distributed.dashboard.components.scheduler - ERROR - 'sub-f27b7da6d2b9a5674b7e1b29b406746b'
Traceback (most recent call last):
  File "c:\Users\geo1k\.lo

#### Prep Input data:
- Read data
- Partition into 0.5x0.5 degree tiles

In [3]:
fergana_gdf = gpd.read_file("../data/Train/Fergana_training_samples.shp")
fergana_gdf = fergana_gdf.set_crs(4326)

In [4]:
orenburg_gdf = gpd.read_file("../data/Train/Orenburg_training_samples.shp")
orenburg_gdf = orenburg_gdf.set_crs(4326)

In [5]:
train_gdf = pd.concat([fergana_gdf, orenburg_gdf], ignore_index=True)
train_gdf.shape

(1000, 3)

In [6]:
train_gdf["lon"] = train_gdf["geometry"].x
train_gdf["lat"] = train_gdf["geometry"].y

#### Search Sentinel-2 scenes for the first partition

In [7]:
point_lon = train_gdf["lon"].iloc[0]
point_lat = train_gdf["lat"].iloc[0]

In [8]:
# Sign into Microsoft planetary computer
stac_url = "https://planetarycomputer.microsoft.com/api/stac/v1"
sentinel2_client = pystac_client.open(
    stac_url,
    modifier=pc.sign_inplace
)

In [9]:
bands = ["B02", "B03", "B04", "B05", "B07", "B08", "B8A", "B11"]

# Query Sentinel-2 items for Fergana region (2020-2024)
search = sentinel2_client.search(
    collections=["sentinel-2-l2a"],
    intersects=dict(type="Point", coordinates=(point_lon, point_lat)),
    datetime="2022-01-01/2024-12-31",
)

items = list(search.items())
print(f"Found {len(items)} Sentinel-2 items")

Found 1174 Sentinel-2 items


#### Build the monthly data cube

In [10]:
datacube = stackstac.stack(
    items,
    bounds_latlon=(point_lon-0.00013, point_lat-0.00013, point_lon+0.00013, point_lat+0.00013),
    assets=bands,
    epsg=4326,
    snap_bounds=False,
)
datacube

<xarray.DataArray 'stackstac-c88f990f31ba96215b8dc4edb6526d2b' (time: 1174,
                                                                band: 8, y: 6,
                                                                x: 3)> Size: 1MB
dask.array<fetch_raster_window, shape=(1174, 8, 6, 3), dtype=float64, chunksize=(1, 1, 6, 3), chunktype=numpy.ndarray>
Coordinates: (12/43)
  * time                                     (time) datetime64[ns] 9kB 2022-0...
    id                                       (time) <U54 254kB 'S2B_MSIL2A_20...
  * band                                     (band) <U3 96B 'B02' ... 'B11'
  * x                                        (x) float64 24B 71.75 71.75 71.75
  * y                                        (y) float64 48B 41.4 41.4 ... 41.4
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    ...                                       ...
    gsd                                      (band) float64 64B 10.0 ... 20.0
    title                                    (band) <U37 1kB 'Band 2 - Blue -...
    common_name                              (band) <U7 224B 'blue' ... 'swir16'
    center_wavelength                        (band) float64 64B 0.49 ... 1.61
    full_width_half_max                      (band) float64 64B 0.098 ... 0.143
    epsg                                     int64 8B 4326
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(np.float64(71.7489502376947...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 71.75|\n| 0.00,-0.00, 41.40|\n| 0.00, 0.00,...
    resolution_xy:  (9.425694899817849e-05, 4.109487249544672e-05)

In [11]:
monthly_datacube = datacube.resample(time="1ME").median()
monthly_datacube = monthly_datacube.assign_coords({"year_month": monthly_datacube["time"].dt.strftime("%Y-%m")})
monthly_datacube

<xarray.DataArray 'stackstac-c88f990f31ba96215b8dc4edb6526d2b' (time: 36,
                                                                band: 8, y: 6,
                                                                x: 3)> Size: 41kB
dask.array<stack, shape=(36, 8, 6, 3), dtype=float64, chunksize=(1, 8, 6, 3), chunktype=numpy.ndarray>
Coordinates: (12/17)
  * band                                     (band) <U3 96B 'B02' ... 'B11'
  * x                                        (x) float64 24B 71.75 71.75 71.75
  * y                                        (y) float64 48B 41.4 41.4 ... 41.4
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    s2:product_type                          <U7 28B 'S2MSI2A'
    instruments                              <U3 12B 'msi'
    ...                                       ...
    common_name                              (band) <U7 224B 'blue' ... 'swir16'
    center_wavelength                        (band) float64 64B 0.49 ... 1.61
    full_width_half_max                      (band) float64 64B 0.098 ... 0.143
    epsg                                     int64 8B 4326
  * time                                     (time) datetime64[ns] 288B 2022-...
    year_month                               (time) object 288B '2022-01' ......
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(np.float64(71.7489502376947...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 71.75|\n| 0.00,-0.00, 41.40|\n| 0.00, 0.00,...
    resolution_xy:  (9.425694899817849e-05, 4.109487249544672e-05)

#### Add agricultural indices

In [12]:
def normalized_difference(b1: xr.DataArray, b2: xr.DataArray, name: str):
    ndi = (b1 - b2) / (b1 + b2)
    ndi.name = name
    return ndi

def calc_evi(nir: xr.DataArray, red: xr.DataArray, blue: xr.DataArray):
    evi = 2.5 * (
        (nir - red) /
        (nir + 6 * red - 7.5 * blue + 1)
    )
    evi.name = "evi"
    return evi.drop_vars("band")

def calc_chlorophyll(red_edge: xr.DataArray, narrow_nir: xr.DataArray, name: str):
    chlorophyll = (narrow_nir - red_edge) / red_edge
    chlorophyll.name = name
    return chlorophyll.drop_vars("band")


In [13]:
blue        = monthly_datacube.sel(band="B02") / 10000
green       = monthly_datacube.sel(band="B03") / 10000
red         = monthly_datacube.sel(band="B04") / 10000
red_edge_5  = monthly_datacube.sel(band="B05") / 10000
red_edge_7  = monthly_datacube.sel(band="B07") / 10000
nir         = monthly_datacube.sel(band="B08") / 10000
narrow_nir  = monthly_datacube.sel(band="B8A") / 10000
swir        = monthly_datacube.sel(band="B11") / 10000

monthly_datacube["ndvi"] = normalized_difference(nir, red, "ndvi")
monthly_datacube["evi"] = calc_evi(nir, red, blue)
monthly_datacube["ndwi"] = normalized_difference(green, nir, "ndwi")
monthly_datacube["ndmi"] = normalized_difference(nir, swir, "ndmi")
monthly_datacube["wet_chlorophyll"] = calc_chlorophyll(red_edge_5, narrow_nir, "wet_chlorophyll")
monthly_datacube["canopy_chlorophyll"] = calc_chlorophyll(red_edge_7, narrow_nir, "canopy_chlorophyll")
monthly_datacube["ndre"] = normalized_difference(nir, red_edge_5, "ndre")

In [14]:
monthly_datacube

<xarray.DataArray 'stackstac-c88f990f31ba96215b8dc4edb6526d2b' (time: 36,
                                                                band: 8, y: 6,
                                                                x: 3)> Size: 41kB
dask.array<stack, shape=(36, 8, 6, 3), dtype=float64, chunksize=(1, 8, 6, 3), chunktype=numpy.ndarray>
Coordinates: (12/24)
  * band                                     (band) <U3 96B 'B02' ... 'B11'
  * x                                        (x) float64 24B 71.75 71.75 71.75
  * y                                        (y) float64 48B 41.4 41.4 ... 41.4
    s2:datatake_type                         <U8 32B 'INS-NOBS'
    s2:product_type                          <U7 28B 'S2MSI2A'
    instruments                              <U3 12B 'msi'
    ...                                       ...
    evi                                      (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    ndwi                                     (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    ndmi                                     (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    wet_chlorophyll                          (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    canopy_chlorophyll                       (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    ndre                                     (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
Attributes:
    spec:           RasterSpec(epsg=4326, bounds=(np.float64(71.7489502376947...
    crs:            epsg:4326
    transform:      | 0.00, 0.00, 71.75|\n| 0.00,-0.00, 41.40|\n| 0.00, 0.00,...
    resolution_xy:  (9.425694899817849e-05, 4.109487249544672e-05)

In [15]:
desired_coords = ["time", "x", "y", "band", "year_month", "ndvi", "evi", "ndwi", "ndmi", "wet_chlorophyll", "canopy_chlorophyll", "ndre"]
cleaned_point_data = monthly_datacube.drop_vars(
        [coord for coord in monthly_datacube.coords if coord not in desired_coords],
        errors="ignore"
)
cleaned_point_data = cleaned_point_data.drop_attrs()
cleaned_point_data

<xarray.DataArray 'stackstac-c88f990f31ba96215b8dc4edb6526d2b' (time: 36,
                                                                band: 8, y: 6,
                                                                x: 3)> Size: 41kB
dask.array<stack, shape=(36, 8, 6, 3), dtype=float64, chunksize=(1, 8, 6, 3), chunktype=numpy.ndarray>
Coordinates:
    year_month          (time) object 288B '2022-01' '2022-02' ... '2024-12'
    ndvi                (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    evi                 (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    ndwi                (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    ndmi                (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    wet_chlorophyll     (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    canopy_chlorophyll  (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
    ndre                (time, y, x) float64 5kB dask.array<chunksize=(1, 6, 3), meta=np.ndarray>
  * x                   (x) float64 24B 71.75 71.75 71.75
  * y                   (y) float64 48B 41.4 41.4 41.4 41.4 41.4 41.4
  * time                (time) datetime64[ns] 288B 2022-01-31 ... 2024-12-31
  * band                (band) <U3 96B 'B02' 'B03' 'B04' ... 'B08' 'B8A' 'B11'

In [16]:
cleaned_point_data.compute()

<xarray.DataArray 'stackstac-c88f990f31ba96215b8dc4edb6526d2b' (time: 36,
                                                                band: 8, y: 6,
                                                                x: 3)> Size: 41kB
array([[[[4664. , 4662. , 4630. ],
         [4644. , 4670. , 4640. ],
         [4640. , 4666. , 4664. ],
         [4666. , 4666. , 4664. ],
         [4674. , 4668. , 4676. ],
         [4674. , 4668. , 4676. ]],

        [[4812. , 4816. , 4832. ],
         [4788. , 4816. , 4840. ],
         [4760. , 4788. , 4812. ],
         [4760. , 4788. , 4812. ],
         [4750. , 4746. , 4782. ],
         [4750. , 4746. , 4782. ]],

        [[4946. , 4910. , 4980. ],
         [4924. , 4934. , 4954. ],
         [4940. , 4950. , 4962. ],
         [4966. , 4950. , 4962. ],
         [4956. , 4938. , 4942. ],
         [4956. , 4938. , 4942. ]],
...
        [[6602. , 7092. , 7812.5],
         [6556. , 6999.5, 8014.5],
         [6750.5, 7158. , 8170. ],
         [6811.5, 7158. , 8170. ],
         [7065.5, 7512. , 7861. ],
         [7065.5, 7512. , 7861. ]],

        [[6473. , 6908.5, 7055.5],
         [6473. , 6908.5, 7055.5],
         [6385.5, 6920. , 7105.5],
         [6372.5, 6920. , 7105.5],
         [6372.5, 6895.5, 7168.5],
         [6372.5, 6895.5, 7168.5]],

        [[2174. , 2204. , 2255.5],
         [2174. , 2204. , 2255.5],
         [2189.5, 2200. , 2282. ],
         [2179.5, 2200. , 2282. ],
         [2179.5, 2234.5, 2296. ],
         [2179.5, 2234.5, 2296. ]]]], shape=(36, 8, 6, 3))
Coordinates:
    year_month          (time) object 288B '2022-01' '2022-02' ... '2024-12'
    ndvi                (time, y, x) float64 5kB 0.0281 0.02945 ... 0.01119
    evi                 (time, y, x) float64 5kB 0.07202 0.07678 ... 0.09633
    ndwi                (time, y, x) float64 5kB -0.04182 -0.03911 ... -0.01139
    ndmi                (time, y, x) float64 5kB 0.3364 0.3345 ... 0.5415 0.5479
    wet_chlorophyll     (time, y, x) float64 5kB 0.01981 0.01645 ... -0.007683
    canopy_chlorophyll  (time, y, x) float64 5kB 0.007552 0.004062 ... -0.008986
    ndre                (time, y, x) float64 5kB 0.02558 0.02188 ... 0.04223
  * x                   (x) float64 24B 71.75 71.75 71.75
  * y                   (y) float64 48B 41.4 41.4 41.4 41.4 41.4 41.4
  * time                (time) datetime64[ns] 288B 2022-01-31 ... 2024-12-31
  * band                (band) <U3 96B 'B02' 'B03' 'B04' ... 'B08' 'B8A' 'B11'

In [ ]:
import pandas as pd

# Load the zarr data
data = xr.open_zarr("part1_2.zarr")

# Reshape to create the wide format
def create_wide_dataframe(data):
    """Convert xarray to wide dataframe with columns for each index-month combination"""

    # Convert to pandas with multi-index columns
    df = data.to_dataframe().unstack(['time', 'band'])

    # Flatten column names to format like 'NDVI_2024-01', 'EVI_2024-01', etc.
    df.columns = [f"{band}_{time.strftime('%Y-%m')}"
                for time, band in df.columns]

    # Reset index to get point_id as a column
    df = df.reset_index()

    # Add the original point metadata (cropland labels, coordinates, etc.)
    df = df.merge(first_part_df[['ID', 'Cropland']],
                left_on='point_id', right_on='ID', how='left')

    return df

# Create the wide dataframe
wide_df = create_wide_dataframe(data)
print(f"DataFrame shape: {wide_df.shape}")
print("Columns:", wide_df.columns.tolist()[:10])  # Show first 10 columns

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Prepare features and target
X = wide_df.drop(['point_id', 'ID', 'Cropland', 'x', 'y'], axis=1)
y = wide_df['Cropland']

# Handle any NaN values
X = X.fillna(X.median())

# Scale features (important for LogReg)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)

# Fit LogisticRegression
lr = LogisticRegression(random_state=42, max_iter=1000)
lr.fit(X_train, y_train)

# Evaluate
y_pred = lr.predict(X_test)
print(classification_report(y_test, y_pred))

In [25]:
client.close()

2025-09-21 16:27:05,217 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21 16:27:05,219 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21 16:27:05,222 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21 16:27:05,224 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21 16:27:05,230 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21 16:27:05,233 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21 16:27:05,239 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21 16:27:05,243 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21 16:27:05,246 - distributed.nanny - WARNING - Worker process still alive after 4.0 seconds, killing
2025-09-21